In [2]:
#import libary for use method
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_style('whitegrid')
import warnings
warnings.filterwarnings('ignore')

import time
import re
import string
from numpy import mean
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest, f_classif

import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold, KFold, GridSearchCV
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix, precision_recall_curve, auc, roc_curve, recall_score, classification_report 
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.utils.multiclass import type_of_target

from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler


In [3]:
#load dataset
df = pd.read_csv('train_Df64byy.csv')
# df.drop('ID',axis='columns', inplace=True)
df

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Response
0,1,C3,3213,Rented,Individual,36,36,No,X1,14+,3.0,22,11628.0,0
1,2,C5,1117,Owned,Joint,75,22,No,X2,NaN,NaN,22,30510.0,0
2,3,C5,3732,Owned,Individual,32,32,No,NaN,1.0,1.0,19,7450.0,1
3,4,C24,4378,Owned,Joint,52,48,No,X1,14+,3.0,19,17780.0,0
4,5,C8,2190,Rented,Individual,44,44,No,X2,3.0,1.0,16,10404.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50877,50878,C4,845,Rented,Individual,22,22,No,X3,NaN,NaN,18,7704.0,0
50878,50879,C5,4188,Rented,Individual,27,27,No,X3,7.0,3.0,4,5408.0,0
50879,50880,C1,442,Rented,Individual,63,63,No,X2,14+,1.0,12,11374.0,0
50880,50881,C1,4,Owned,Joint,71,49,No,X2,2.0,2.0,16,28179.2,1


In [4]:
#load dataset
dft = pd.read_csv('test_YCcRUnU.csv')
# dft.drop('ID',axis='columns', inplace=True)
# id=dft['ID']
dft

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium
0,50883,C1,156,Owned,Individual,30,30,No,NaN,6.0,3.0,5,11934.0
1,50884,C4,7,Owned,Joint,69,68,Yes,X1,3.0,3.0,18,32204.8
2,50885,C1,564,Rented,Individual,28,28,No,X3,2.0,4.0,17,9240.0
3,50886,C3,1177,Rented,Individual,23,23,No,X3,3.0,3.0,18,9086.0
4,50887,C1,951,Owned,Individual,75,75,No,X3,NaN,NaN,5,22534.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21800,72683,C3,1044,Owned,Individual,45,45,No,X1,4.0,1.0,18,15884.0
21801,72684,C4,266,Owned,Individual,59,59,No,X5,6.0,3.0,18,21390.0
21802,72685,C12,2470,Owned,Individual,74,74,No,X3,NaN,NaN,1,17836.0
21803,72686,C10,1676,Rented,Individual,25,25,No,X4,3.0,1.0,19,11568.0


In [5]:
print('Train Data Shape: ', df.shape)
print('Test Data Shape: ', dft.shape)

Train Data Shape:  (50882, 14)
Test Data Shape:  (21805, 13)


In [6]:
dft.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21805 entries, 0 to 21804
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       21805 non-null  int64  
 1   City_Code                21805 non-null  object 
 2   Region_Code              21805 non-null  int64  
 3   Accomodation_Type        21805 non-null  object 
 4   Reco_Insurance_Type      21805 non-null  object 
 5   Upper_Age                21805 non-null  int64  
 6   Lower_Age                21805 non-null  int64  
 7   Is_Spouse                21805 non-null  object 
 8   Health Indicator         16778 non-null  object 
 9   Holding_Policy_Duration  13202 non-null  object 
 10  Holding_Policy_Type      13202 non-null  float64
 11  Reco_Policy_Cat          21805 non-null  int64  
 12  Reco_Policy_Premium      21805 non-null  float64
dtypes: float64(2), int64(5), object(6)
memory usage: 2.2+ MB


In [7]:
# Checking null values 
null_columns=dft.columns[dft.isnull().any()]
dft[null_columns].isnull().sum()

Health Indicator           5027
Holding_Policy_Duration    8603
Holding_Policy_Type        8603
dtype: int64

In [8]:
Null_columns=dft.select_dtypes(include=['object','float64']).columns.tolist()

for column in dft:
    if dft[column].isnull().any():
        if(column in Null_columns):
            dft[column]=dft[column].fillna(dft[column].mode()[0])

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50882 entries, 0 to 50881
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       50882 non-null  int64  
 1   City_Code                50882 non-null  object 
 2   Region_Code              50882 non-null  int64  
 3   Accomodation_Type        50882 non-null  object 
 4   Reco_Insurance_Type      50882 non-null  object 
 5   Upper_Age                50882 non-null  int64  
 6   Lower_Age                50882 non-null  int64  
 7   Is_Spouse                50882 non-null  object 
 8   Health Indicator         39191 non-null  object 
 9   Holding_Policy_Duration  30631 non-null  object 
 10  Holding_Policy_Type      30631 non-null  float64
 11  Reco_Policy_Cat          50882 non-null  int64  
 12  Reco_Policy_Premium      50882 non-null  float64
 13  Response                 50882 non-null  int64  
dtypes: float64(2), int64(6

In [10]:
# Checking null values 
null_columns=df.columns[df.isnull().any()]
df[null_columns].isnull().sum()

Health Indicator           11691
Holding_Policy_Duration    20251
Holding_Policy_Type        20251
dtype: int64

In [11]:
null_columns=df.select_dtypes(include=['object','float64']).columns.tolist()

for column in df:
    if df[column].isnull().any():
        if(column in null_columns):
            df[column]=df[column].fillna(df[column].mode()[0])

In [12]:
list(df.select_dtypes(exclude='object').columns)

['ID',
 'Region_Code',
 'Upper_Age',
 'Lower_Age',
 'Holding_Policy_Type',
 'Reco_Policy_Cat',
 'Reco_Policy_Premium',
 'Response']

In [13]:
df.select_dtypes(['object']).columns

Index(['City_Code', 'Accomodation_Type', 'Reco_Insurance_Type', 'Is_Spouse',
       'Health Indicator', 'Holding_Policy_Duration'],
      dtype='object')

In [14]:
columns_to_encode = ['City_Code', 'Accomodation_Type', 'Reco_Insurance_Type', 'Is_Spouse',
       'Health Indicator', 'Holding_Policy_Duration']
numerical_cols = ['Region_Code',
 'Upper_Age',
 'Lower_Age',
 'Holding_Policy_Type',
 'Reco_Policy_Cat',
 'Reco_Policy_Premium']

In [15]:
from sklearn.preprocessing import LabelEncoder
for names in columns_to_encode:
    Label_enc = LabelEncoder()
    df[names] = Label_enc.fit_transform(df[names].values)

In [16]:
from sklearn.preprocessing import LabelEncoder
for names in columns_to_encode:
    Label_enc = LabelEncoder()
    dft[names] = Label_enc.fit_transform(dft[names].values)

In [17]:
#Lets bring input dataset features to same scale
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
df[numerical_cols] = ss.fit_transform(df[numerical_cols])
dft[numerical_cols] = ss.fit_transform(dft[numerical_cols])

In [18]:
train_data = df.copy()
test_data = dft.copy()

train_data = train_data.drop(['ID'], axis = 1)
train_data['Response'] = train_data['Response'].apply(lambda x: np.int(x))

testIDs = test_data['ID']
test_data = test_data.drop(['ID'], axis = 1)
train_data.head()

,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Response
0,22,1.039425,1,0,-0.511625,-0.389098,0,0,5,0.400939,1.085830,-0.387852,0
1,31,-0.432415,0,1,1.741395,-1.197449,0,1,0,0.400939,1.085830,2.477394,0
2,31,1.403874,0,0,-0.742704,-0.620055,0,0,0,-1.974398,0.612688,-1.021842,1
3,16,1.857504,0,1,0.412691,0.303775,0,0,5,0.400939,0.612688,0.545682,0
4,34,0.321060,1,0,-0.049467,0.072817,0,1,8,-1.974398,0.139547,-0.573588,0


In [19]:
#Lets divide the dataset into input and output
X = train_data.drop(['Response'], axis = 1)
y = train_data["Response"]

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3,random_state=10)

In [21]:
#Decision tree

from sklearn.tree import DecisionTreeClassifier
model_DecisionTree=DecisionTreeClassifier(criterion='gini',random_state=10)
model_DecisionTree.fit(X_train,Y_train)
Y_pred=model_DecisionTree.predict(X_test)

In [22]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

cfm=confusion_matrix(Y_test,Y_pred)
print(cfm)

print()

print("classification report:")
print(classification_report(Y_test,Y_pred))

print()

auc=roc_auc_score(Y_test, Y_pred)
print("Auc of the model: ",auc)

[[8962 2648]
 [2474 1181]]

classification report:
              precision    recall  f1-score   support

           0       0.78      0.77      0.78     11610
           1       0.31      0.32      0.32      3655

    accuracy                           0.66     15265
   macro avg       0.55      0.55      0.55     15265
weighted avg       0.67      0.66      0.67     15265


Auc of the model:  0.5475198865075745


In [23]:
print(list(zip(df.columns,model_DecisionTree.feature_importances_)))

[('ID', 0.09549501937969476), ('City_Code', 0.24957250412778587), ('Region_Code', 0.020102055520819683), ('Accomodation_Type', 0.004714984429853489), ('Reco_Insurance_Type', 0.09481695003082106), ('Upper_Age', 0.10005703479804805), ('Lower_Age', 0.008122770808098706), ('Is_Spouse', 0.05748182111961392), ('Health Indicator', 0.07134960182892726), ('Holding_Policy_Duration', 0.02282498464918452), ('Holding_Policy_Type', 0.0536991849594066), ('Reco_Policy_Cat', 0.22176308834774597)]


In [24]:
model_DecisionTree.get_depth()

40

In [25]:
# Pruning DT

from sklearn.tree import DecisionTreeClassifier
model_DecisionTree=DecisionTreeClassifier(criterion='gini',min_samples_leaf=5,max_depth=25,random_state=10)
model_DecisionTree.fit(X_train,Y_train)
Y_pred=model_DecisionTree.predict(X_test)

In [26]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

cfm=confusion_matrix(Y_test,Y_pred)
print(cfm)

print()

print("classification report:")
print(classification_report(Y_test,Y_pred))

print()

auc=roc_auc_score(Y_test, Y_pred)
print("Auc of the model: ",auc)

[[9649 1961]
 [2658  997]]

classification report:
              precision    recall  f1-score   support

           0       0.78      0.83      0.81     11610
           1       0.34      0.27      0.30      3655

    accuracy                           0.70     15265
   macro avg       0.56      0.55      0.55     15265
weighted avg       0.68      0.70      0.69     15265


Auc of the model:  0.5519354511830572


In [27]:
#Random Forest

from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier(n_estimators=200,random_state=10)
model.fit(X_train,Y_train)
Y_pred=model.predict(X_test)

In [28]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

cfm=confusion_matrix(Y_test,Y_pred)
print(cfm)

print()

print("classification report:")
print(classification_report(Y_test,Y_pred))

print()

auc=roc_auc_score(Y_test, Y_pred)
print("Auc of the model: ",auc)

[[11466   144]
 [ 3557    98]]

classification report:
              precision    recall  f1-score   support

           0       0.76      0.99      0.86     11610
           1       0.40      0.03      0.05      3655

    accuracy                           0.76     15265
   macro avg       0.58      0.51      0.46     15265
weighted avg       0.68      0.76      0.67     15265


Auc of the model:  0.507204742362061


In [29]:
#AdaBoost

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
model_AdaBoost=AdaBoostClassifier(base_estimator=DecisionTreeClassifier(random_state=10),n_estimators=50,random_state=10)
model_AdaBoost.fit(X_train,Y_train)
Y_pred=model_AdaBoost.predict(X_test)

In [30]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

cfm=confusion_matrix(Y_test,Y_pred)
print(cfm)

print()

print("classification report:")
print(classification_report(Y_test,Y_pred))

print()

auc=roc_auc_score(Y_test, Y_pred)
print("Auc of the model: ",auc)

[[8976 2634]
 [2446 1209]]

classification report:
              precision    recall  f1-score   support

           0       0.79      0.77      0.78     11610
           1       0.31      0.33      0.32      3655

    accuracy                           0.67     15265
   macro avg       0.55      0.55      0.55     15265
weighted avg       0.67      0.67      0.67     15265


Auc of the model:  0.551953184374525


In [31]:
#gradient boost

from sklearn.ensemble import GradientBoostingClassifier
model=GradientBoostingClassifier(n_estimators=150,random_state=10)
model.fit(X_train,Y_train)
Y_pred=model.predict(X_test)

In [32]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

cfm=confusion_matrix(Y_test,Y_pred)
print(cfm)

print()

print("classification report:")
print(classification_report(Y_test,Y_pred))

print()

auc=roc_auc_score(Y_test, Y_pred)
print("Auc of the model: ",auc)

[[11594    16]
 [ 3644    11]]

classification report:
              precision    recall  f1-score   support

           0       0.76      1.00      0.86     11610
           1       0.41      0.00      0.01      3655

    accuracy                           0.76     15265
   macro avg       0.58      0.50      0.43     15265
weighted avg       0.68      0.76      0.66     15265


Auc of the model:  0.5008157268075188


In [33]:
#SVM 

from sklearn import svm
classifier=svm.SVC(kernel="rbf", gamma=0.1, C=50)
classifier.fit(X_train,Y_train)
Y_pred=classifier.predict(X_test)

In [34]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

cfm=confusion_matrix(Y_test,Y_pred)
print(cfm)

print()

print("classification report:")
print(classification_report(Y_test,Y_pred))

print()

auc=roc_auc_score(Y_test, Y_pred)
print("Accuracy of the model: ",auc)

[[10330  1280]
 [ 3176   479]]

classification report:
              precision    recall  f1-score   support

           0       0.76      0.89      0.82     11610
           1       0.27      0.13      0.18      3655

    accuracy                           0.71     15265
   macro avg       0.52      0.51      0.50     15265
weighted avg       0.65      0.71      0.67     15265


Accuracy of the model:  0.5104017834523991


In [35]:
#Logistic 

from sklearn.linear_model import LogisticRegression
classifier=LogisticRegression()
classifier.fit(X_train,Y_train)
Y_pred=classifier.predict(X_test)

In [36]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

cfm=confusion_matrix(Y_test,Y_pred)
print(cfm)

print()

print("classification report:")
print(classification_report(Y_test,Y_pred))

print()

auc=roc_auc_score(Y_test, Y_pred)
print("Auc of the model: ",auc)

[[11610     0]
 [ 3655     0]]

classification report:
              precision    recall  f1-score   support

           0       0.76      1.00      0.86     11610
           1       0.00      0.00      0.00      3655

    accuracy                           0.76     15265
   macro avg       0.38      0.50      0.43     15265
weighted avg       0.58      0.76      0.66     15265


Auc of the model:  0.5


In [37]:

#Ensembled Model 

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier

# create the sub models
estimators = []
model1 = LogisticRegression()
estimators.append(('log', model1))
model2 = DecisionTreeClassifier(criterion='gini',min_samples_leaf=5,max_depth=25,random_state=10)
estimators.append(('cart', model2))
model4=RandomForestClassifier(n_estimators=200,random_state=10)
estimators.append(('RandomForest', model4))
model5=GradientBoostingClassifier(n_estimators=150,random_state=10)
estimators.append(('GradientBoost', model5))

# create the ensemble model
ensemble = VotingClassifier(estimators)
ensemble.fit(X_train,Y_train)
Y_pred=ensemble.predict(X_test)

In [38]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

cfm=confusion_matrix(Y_test,Y_pred)
print(cfm)

print()

print("classification report:")
print(classification_report(Y_test,Y_pred))

print()

auc=roc_auc_score(Y_test, Y_pred)
print("Auc of the model: ",auc)

[[11607     3]
 [ 3653     2]]

classification report:
              precision    recall  f1-score   support

           0       0.76      1.00      0.86     11610
           1       0.40      0.00      0.00      3655

    accuracy                           0.76     15265
   macro avg       0.58      0.50      0.43     15265
weighted avg       0.67      0.76      0.66     15265


Auc of the model:  0.5001443988448092


In [39]:
#final prediction on best auc scored model.
final_pred = model_AdaBoost.predict(test_data)
submissionC = pd.DataFrame(data = {'ID': testIDs, 'Response': final_pred})
submissionC.to_csv("final_result.csv", index = False)
submissionC.head()

,ID,Response
0,50883,0
1,50884,1
2,50885,0
3,50886,0
4,50887,0


In [ ]:
# AdaBoostClassifier with DecisionTreeClassifier gives Auc of the model:  0.551953184374525